In [1]:
import geopandas as gpd

blocks_land_use_gdf = gpd.read_parquet('./../data/blocks.parquet')[['geometry', 'land_use']]
blocks_land_use_gdf.head()

,geometry,land_use
id,,
0,"POLYGON ((354918.622 6625258.829, 354901.464 6...",None
1,"POLYGON ((355412.142 6623378.149, 355411.700 6...",transport
2,"POLYGON ((353934.329 6625429.433, 353923.453 6...",recreation
3,"POLYGON ((355099.099 6623847.765, 355074.808 6...",residential
4,"POLYGON ((352766.168 6621954.748, 352744.412 6...",recreation


In [2]:
from blocksnet.preprocessing.feature_engineering import generate_geometry_features

blocks_features_gdf = generate_geometry_features(blocks_land_use_gdf, aspect_ratios=True)
blocks_features_gdf.head()

2025-03-23 01:01:11.061 | INFO     | blocksnet.preprocessing.feature_engineering.core:_calculate_usual_features:34 - Calculating usual features.
2025-03-23 01:01:11.495 | INFO     | blocksnet.preprocessing.feature_engineering.core:_calculate_aspect_ratios:57 - Calculating radiuses.
100%|██████████| 16320/16320 [00:00<00:00, 28929.12it/s]


,geometry,x,y,area,length,corners_count,aspect_ratio
id,,,,,,,
0,"POLYGON ((354918.622 6625258.829, 354901.464 6...",354523.996322,6.625731e+06,8.044667e+05,6143.314507,180,2.194869
1,"POLYGON ((355412.142 6623378.149, 355411.700 6...",355410.584682,6.623480e+06,2.317313e+04,1305.400332,53,15.785382
2,"POLYGON ((353934.329 6625429.433, 353923.453 6...",353810.130041,6.625813e+06,3.630058e+05,2744.750098,88,1.107639
3,"POLYGON ((355099.099 6623847.765, 355074.808 6...",355194.637468,6.624275e+06,1.964145e+05,2588.389797,77,2.632711
4,"POLYGON ((352766.168 6621954.748, 352744.412 6...",353157.203348,6.622549e+06,1.781752e+06,5826.501550,156,1.864688


In [3]:
blocks_gdf = blocks_features_gdf.join(blocks_land_use_gdf[['land_use']])
blocks_gdf.head()

,geometry,x,y,area,length,corners_count,aspect_ratio,land_use
id,,,,,,,,
0,"POLYGON ((354918.622 6625258.829, 354901.464 6...",354523.996322,6.625731e+06,8.044667e+05,6143.314507,180,2.194869,None
1,"POLYGON ((355412.142 6623378.149, 355411.700 6...",355410.584682,6.623480e+06,2.317313e+04,1305.400332,53,15.785382,transport
2,"POLYGON ((353934.329 6625429.433, 353923.453 6...",353810.130041,6.625813e+06,3.630058e+05,2744.750098,88,1.107639,recreation
3,"POLYGON ((355099.099 6623847.765, 355074.808 6...",355194.637468,6.624275e+06,1.964145e+05,2588.389797,77,2.632711,residential
4,"POLYGON ((352766.168 6621954.748, 352744.412 6...",353157.203348,6.622549e+06,1.781752e+06,5826.501550,156,1.864688,recreation


In [4]:
from blocksnet.relations import generate_adjacency_graph, get_adjacency_context

adjacency_graph = generate_adjacency_graph(blocks_land_use_gdf)

2025-03-23 01:01:12.129 | INFO     | blocksnet.relations.adjacency.core:_generate_adjacency_nodes:10 - Generating nodes.
2025-03-23 01:01:12.137 | INFO     | blocksnet.relations.adjacency.core:_generate_adjacency_edges:15 - Generating edges.
2025-03-23 01:01:12.729 | SUCCESS  | blocksnet.relations.adjacency.core:generate_adjacency_graph:34 - Adjacency graph successfully generated: 16320 nodes, 15920 edges


## Optimization

In [5]:
from blocksnet.enums import LandUse

target_shares = {
    LandUse.RESIDENTIAL : 0.5,
    LandUse.RECREATION : 0.3,
    LandUse.BUSINESS : 0.2
}

blocks_ids = list(blocks_gdf.head(100).index)

In [ ]:
from blocksnet.optimization.land_use import LandUseOptimizer

luo = LandUseOptimizer(blocks_gdf, adjacency_graph)
res_df = luo.run(blocks_ids, target_shares, population_size=10, n_generations=10, mutation_probability=0.5)

n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |      100 |      1 |             - |             -
     2 |      200 |      1 |  0.0150899949 |         ideal
     3 |      300 |      1 |  0.000000E+00 |             f
     4 |      400 |      1 |  0.0053634371 |         ideal
     5 |      500 |      1 |  0.0019405345 |             f
     6 |      600 |      1 |  0.0019405345 |             f
     7 |      700 |      1 |  0.0122393004 |         ideal
     8 |      800 |      1 |  0.000000E+00 |             f
     9 |      900 |      1 |  0.000000E+00 |             f
    10 |     1000 |      1 |  0.000000E+00 |             f
    11 |     1100 |      1 |  0.000000E+00 |             f
    12 |     1200 |      1 |  0.000000E+00 |             f
    13 |     1300 |      1 |  0.000000E+00 |             f
    14 |     1400 |      1 |  0.0014374359 |             f
    15 |     1500 |      1 |  0.0014374359 |             f
    16 |     1600 |      1 |  0.0014374359 |            

In [11]:
res_df

,solution,objectives,assigned_land_use,objective_1,objective_2
0,"[0, 4, 2, 1, 2, 5, 2, 2, 6, 4, 6, 5, 1, 0, 0, ...","[0.08386083142477235, 0.9910495503978706]","{0: LandUse.RESIDENTIAL, 1: LandUse.TRANSPORT,...",0.083861,0.99105
